En este notebook vamos a probar la clase hecha para el proyecto final. Voy a definir la función de las cámaras.

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as la

In [92]:
class Optimizador:
    epsilon = 0.00001
    max_iter = 100000
    def __init__(self, epsilon = 0.00001, max_iter = 10000):
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.printInstrucciones()
    def printInstrucciones(self):
        print('Para optimizar una función usa el método optimiza, donde f representa la función a optimizar, dimensión\nrepresenta el tamaño del dominio de f, y Tipo representa el método a utilizar.')

        print('Tipo == 0 utilizará BFGS')
        print('Tipo == 1 utilizará una búsqueda lineal base')
        print('Tipo == 2 utilizará el método de Newton con modificación a la Hessiana')
        print('Tipo == 3 utilizará el método de Newton')
    def optimiza(self, f, dimension, tipo):
        x = np.zeros(dimension)
        if tipo == 0 : # BFGS
            return self.BFGS(f,x)
        elif tipo == 1: #Line Base
            return self.newton_line_base(f,x)
        elif tipo == 2: #Newton with modification
            return self.newton_mH(f,x)
        elif tipo == 3:
            return self.newton(f,x)
        else:
            print('\nTipo no valido')
            self.printInstrucciones()
    def derivada_parcial(self,f, xk, pos):
        eps = 0.0001
        n = xk.size
        h = np.zeros(n)
        h[pos] += eps
        return (f(xk + h) - f(xk)) / eps

    def derivada(self, f, xk):
        return (f(xk + self.epsilon) - f(xk))/self.epsilon

    def gradiente(self, f, xk):
        n = xk.size
        res = np.zeros(n)
        for i in range(n):
            res[i] = self.derivada_parcial(f, xk, i)
        return res

    def segunda_derivada(self, f, xk, pos1, pos2):
        n = xk.size
        h = np.zeros(n)
        h[pos2] += self.epsilon

        def f_prima(x):
            return self.derivada_parcial(f, x, pos1)

        return self.derivada_parcial(f_prima, xk, pos2)

    def hessiana(self, f, xk):
        n = xk.size
        res = np.zeros((n, n))
        for i in range(n):
            for j in range(i, n):
                res[i][j] = self.segunda_derivada(f, xk, i, j)
                res[j][i] = res[i][j]
        return res

    def is_pos_def(self, H):
        return np.all(np.linalg.eigvals(H) > self.epsilon)

    def condiciones_optimalidad(self, f, xk):
        if (np.all(self.gradiente(f, xk) >= self.epsilon)):
            return self.is_pos_def(self.hessiana(f, xk))
        return False

    def mk(self, f, xk, p):
        pt = p.transpose()
        return f(xk) + pt.dot(self.gradiente(f, xk)) + .5 * (pt.dot((self.hessiana(f, xk)).dot(p)))


    def backtracking_line_search(self, f, xk, p, c=.001, ro=.5):
        alpha = 1;
        f_k = f(xk)
        gr = self.gradiente(f, xk)
        while f(xk + alpha * p) > f_k + c * alpha * gr.dot(p):
            alpha *= ro;
        return alpha
    def make_pos_def(self,H):
        n = H.shape[0]
        Id = np.identity(n)
        t = self.epsilon
        while(not self.is_pos_def(H)):
            H = H + t*Id
            t= 2*t
        return H

    def zoom(self, a_low, a_high, phi, num=0, c1=0.0001, c2=0.9):
        a_mid = (a_high + a_low)/2
        if 20 < num:
            return a_mid
        if(phi(0) + c1*a_mid*(self.derivada(phi,0)) < phi(a_mid)) or (phi(a_low) <= phi(a_mid)):
            return self.zoom(a_low, a_mid, phi, num+1)
        else:
            if abs(self.derivada(phi, a_mid)) <= (-1)*c2*self.derivada(phi,0):
                return a_mid
            if 0 <= self.derivada(phi, a_mid)*(a_high - a_low):
                a_high = a_low
            return self.zoom(a_mid, a_high, phi, num+1)


    def BFGS(self, f, xk):
        #tomamos H0 simplemente como la identidad, como sugiere el Nocedal.
        n = xk.size
        Id = np.identity(n)
        Hk = Id
        gr = self.gradiente(f,xk)
        count = 0
        while self.epsilon < la.norm(gr) and count < self.max_iter:
            count = count +1
            pk = (-1)*Hk.dot(gr)
            al = self.backtracking_line_search(f,xk,pk)
            xk_n = xk + al*pk
            gr_n = self.gradiente(f,xk_n)
            sk = np.array([xk_n - xk])
            yk = np.array([gr_n-gr]) #hago yk una matriz de n*1
            yk = yk.T #transpongo para que sean vector columna
            sk = sk.T
            if ((yk.T).dot(sk))[0][0] < self.epsilon:
                ro = 1/self.epsilon
            else:
                ro = 1 /(((yk.T).dot(sk))[0][0])
            Hk = (Id + ((-1)*ro*sk).dot(yk.T)).dot(Hk).dot(Id + ((-1)*ro*yk).dot(sk.T)) + (ro*sk).dot(sk.T)
            xk = xk_n
            gr = gr_n
        return xk



    def newton_mH(self, f, x):
        eps = 0.0001
        n = 1000
        for i in range(self.max_iter):
            B = self.make_pos_def(self.hessiana(f, x))
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)

            alpha = self.backtracking_line_search(f, x, p)
            x += alpha * p
        return x

    def newton (self, f, x):
        for i in range(self.max_iter):
            B = self.hessiana(f, x)
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)
            alpha = self.backtracking_line_search(f, x, p)
            x += alpha * p
        return x

    def line_base(self, f, xk, p, c1=0.0001, c2=0.9):
        def phi(al):
           return f(xk + al*p)
        al1 = 0
        al_max = 100
        al2 = al_max /2
        for i in range (50):
            if (phi(0) + c1*al2*self.derivada(phi,0) < phi(al2))  or  (0 < i and phi(al1) <= phi(al2)):
                return self.zoom(al1, al2, phi)
            if abs(self.derivada(phi, al2)) <= (-1)*c2*self.derivada(phi,0):
                return al2
            if(0 <= self.derivada(phi,al2)):
                return self.zoom(al2, al1, phi)
            al1 = al2
            al2 = (al_max + al1)/2
        return al2

    def newton_line_base(self, f, x):
        eps = 0.0001
        n = 1000
        for i in range(self.max_iter):
            B = self.hessiana(f, x)
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)
            alpha = self.line_base(f, x, p)
            x += alpha * p
        return x


In [93]:
op = Optimizador()

Para optimizar una función usa el método optimiza, donde f representa la función a optimizar, dimensión
representa el tamaño del dominio de f, y Tipo representa el método a utilizar.
Tipo == 0 utilizará BFGS
Tipo == 1 utilizará una búsqueda lineal base
Tipo == 2 utilizará el método de Newton con modificación a la Hessiana
Tipo == 3 utilizará el método de Newton


Ahora vamos a probar la clase con la función de Rosenbrock con distintos parámetros y obviamente con los distinos métodos.

In [94]:
def rosenbrock(x, a = 1, b = 100):
    # x pertenece a R2
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2;

## BFGS

In [95]:
op.optimiza(rosenbrock,2,0)

array([0.97136253, 0.94349515])

Aquí vemos que se acerca bastante al resultado que esperabamos que era 1,1. Por lo tanto, BFGS funciona bien.

Cambiamos la función de Rosenbrock para el siguiente método

## Búsqueda lineal base

In [98]:
def rosenbrock(x, a = 1.5, b = 20):
    # x pertenece a R2
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2;

In [99]:
op.optimiza(rosenbrock,2,1)

array([1.48813525, 2.21460945])

De igual manera, vemos que la Busqueda Lineal Base también funciona muy bien, el resultado que esperabamos era 1.5, 2.25.

Volvemos a cambiar para probar Newton

## Newton con modificación a la Hessiana

In [101]:
def rosenbrock(x, a = 2, b = 25):
    # x pertenece a R2
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2;

In [102]:
op.optimiza(rosenbrock,2,2)

array([1.9754972 , 3.90253918])

Vemos que también el método de Newton con modificación a la Hessiana funciona correctamente, como lo esperabamos, aquí queríamos 2,4 como resultado

## Newton

In [104]:
def rosenbrock(x, a =5, b = 10):
    # x pertenece a R2
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2;

In [105]:
op.optimiza(rosenbrock,2,3)

array([ 4.94607556, 24.46361343])

Por último, el método de Newton sin modificar la Hessiana es un poco más inexacto pero también aproxima la solución bastante bien, ya que esperabamos 5,25

Ahora sí, la prueba de fuego, el problema de las cámaras.

In [2]:
crime= pd.read_csv("crime_data.csv")

In [3]:
crime.head()

,crime,date,hour,lat,long
0,VIOLACION,2020-09-30,07:20,19.318714,-99.254418
1,VIOLACION,2020-09-29,22:30,19.378762,-99.205488
2,VIOLACION,2020-09-29,22:00,19.347971,-99.030162
3,VIOLACION,2020-09-29,23:30,19.557311,-99.134166
4,VIOLACION,2020-09-27,00:00,19.342125,-99.124280


Voy a quitar las columnas que no me sirven.

In [5]:
crime.drop(['crime', 'date', 'hour'], axis = 1, inplace = True)

In [8]:
crime.shape

(31056, 2)

Hago una pequeña modificación en la clase para que se calcule de manera más rápida el problema de las cámaras

In [ ]:
class Optimizador:
    epsilon = 0.00001
    max_iter = 10
    def __init__(self, epsilon = 0.00001, max_iter = 100):
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.printInstrucciones()
    def printInstrucciones(self):
        print('Para optimizar una función usa el método optimiza, donde f representa la función a optimizar, dimensión\nrepresenta el tamaño del dominio de f, y Tipo representa el método a utilizar.')

        print('Tipo == 0 utilizará BFGS')
        print('Tipo == 1 utilizará una búsqueda lineal base')
        print('Tipo == 2 utilizará el método de Newton con modificación a la Hessiana')
        print('Tipo == 3 utilizará el método de Newton')
    def optimiza(self, f, dimension, tipo):
        x = np.zeros(dimension)
        if tipo == 0 : # BFGS
            return self.BFGS(f,x)
        elif tipo == 1: #Line Base
            return self.newton_line_base(f,x)
        elif tipo == 2: #Newton with modification
            return self.newton_mH(f,x)
        elif tipo == 3:
            return self.newton(f,x)
        else:
            print('\nTipo no valido')
            self.printInstrucciones()
    def derivada_parcial(self,f, xk, pos):
        eps = 0.0001
        n = xk.size
        h = np.zeros(n)
        h[pos] += eps
        return (f(xk + h) - f(xk)) / eps

    def derivada(self, f, xk):
        return (f(xk + self.epsilon) - f(xk))/self.epsilon

    def gradiente(self, f, xk):
        n = xk.size
        res = np.zeros(n)
        for i in range(n):
            res[i] = self.derivada_parcial(f, xk, i)
        return res

    def segunda_derivada(self, f, xk, pos1, pos2):
        n = xk.size
        h = np.zeros(n)
        h[pos2] += self.epsilon

        def f_prima(x):
            return self.derivada_parcial(f, x, pos1)

        return self.derivada_parcial(f_prima, xk, pos2)

    def hessiana(self, f, xk):
        n = xk.size
        res = np.zeros((n, n))
        for i in range(n):
            for j in range(i, n):
                res[i][j] = self.segunda_derivada(f, xk, i, j)
                res[j][i] = res[i][j]
        return res

    def is_pos_def(self, H):
        return np.all(np.linalg.eigvals(H) > self.epsilon)

    def condiciones_optimalidad(self, f, xk):
        if (np.all(self.gradiente(f, xk) >= self.epsilon)):
            return self.is_pos_def(self.hessiana(f, xk))
        return False

    def mk(self, f, xk, p):
        pt = p.transpose()
        return f(xk) + pt.dot(self.gradiente(f, xk)) + .5 * (pt.dot((self.hessiana(f, xk)).dot(p)))


    def backtracking_line_search(self, f, xk, p, c=.001, ro=.5):
        alpha = 1;
        f_k = f(xk)
        gr = self.gradiente(f, xk)
        while f(xk + alpha * p) > f_k + c * alpha * gr.dot(p):
            alpha *= ro;
        return alpha
    def make_pos_def(self,H):
        n = H.shape[0]
        Id = np.identity(n)
        t = self.epsilon
        while(not self.is_pos_def(H)):
            H = H + t*Id
            t= 2*t
        return H

    def zoom(self, a_low, a_high, phi, num=0, c1=0.0001, c2=0.9):
        a_mid = (a_high + a_low)/2
        if 20 < num:
            return a_mid
        if(phi(0) + c1*a_mid*(self.derivada(phi,0)) < phi(a_mid)) or (phi(a_low) <= phi(a_mid)):
            return self.zoom(a_low, a_mid, phi, num+1)
        else:
            if abs(self.derivada(phi, a_mid)) <= (-1)*c2*self.derivada(phi,0):
                return a_mid
            if 0 <= self.derivada(phi, a_mid)*(a_high - a_low):
                a_high = a_low
            return self.zoom(a_mid, a_high, phi, num+1)


    def BFGS(self, f, xk):
        #tomamos H0 simplemente como la identidad, como sugiere el Nocedal.
        n = xk.size
        Id = np.identity(n)
        Hk = Id
        gr = self.gradiente(f,xk)
        count = 0
        while self.epsilon < la.norm(gr) and count < self.max_iter:
            count = count +1
            pk = (-1)*Hk.dot(gr)
            al = self.backtracking_line_search(f,xk,pk)
            xk_n = xk + al*pk
            gr_n = self.gradiente(f,xk_n)
            sk = np.array([xk_n - xk])
            yk = np.array([gr_n-gr]) #hago yk una matriz de n*1
            yk = yk.T #transpongo para que sean vector columna
            sk = sk.T
            if ((yk.T).dot(sk))[0][0] < self.epsilon:
                ro = 1/self.epsilon
            else:
                ro = 1 /(((yk.T).dot(sk))[0][0])
            Hk = (Id + ((-1)*ro*sk).dot(yk.T)).dot(Hk).dot(Id + ((-1)*ro*yk).dot(sk.T)) + (ro*sk).dot(sk.T)
            xk = xk_n
            gr = gr_n
            if count%10 == 0:
                print('ok')
        return xk



    def newton_mH(self, f, x):
        eps = 0.0001
        n = 1000
        for i in range(self.max_iter):
            B = self.make_pos_def(self.hessiana(f, x))
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)

            alpha = self.backtracking_line_search(f, x, p)
            x += alpha * p
        return x

    def newton (self, f, x):
        for i in range(self.max_iter):
            B = self.hessiana(f, x)
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)
            alpha = self.backtracking_line_search(f, x, p)
            x += alpha * p
        return x

    def line_base(self, f, xk, p, c1=0.0001, c2=0.9):
        def phi(al):
           return f(xk + al*p)
        al1 = 0
        al_max = 100
        al2 = al_max /2
        for i in range (50):
            if (phi(0) + c1*al2*self.derivada(phi,0) < phi(al2))  or  (0 < i and phi(al1) <= phi(al2)):
                return self.zoom(al1, al2, phi)
            if abs(self.derivada(phi, al2)) <= (-1)*c2*self.derivada(phi,0):
                return al2
            if(0 <= self.derivada(phi,al2)):
                return self.zoom(al2, al1, phi)
            al1 = al2
            al2 = (al_max + al1)/2
        return al2

    def newton_line_base(self, f, x):
        eps = 0.0001
        n = 1000
        for i in range(self.max_iter):
            B = self.hessiana(f, x)
            gr = self.gradiente(f, x)
            p = np.linalg.solve(B, -1 * gr)
            alpha = self.line_base(f, x, p)
            x += alpha * p
        return x



In [ ]:
o = Optimizador()

In [9]:
def distancia(x1, y1, x2, y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

Defino la función de las cámaras tal y como expliqué en el ReadMe, nótese que x[2i] es la latitud de la iésima cámara, así como x[2i +1] es la longitud de la iésima cámara.

In [71]:
def camara(x, pen = 5, R = 20):
    
    res = 0
    i = 0
    for i in range(10):
        for j in range(50):
            res = res + distancia(x[2*i],x[2*i+1], crime['lat'][j], crime['long'][j])
    for i in range(10):
        for j in range(i+1, 10):
            dif = R - distancia(x[2*i], x[2*i +1], x[2*j], x[2*j +1])
            if 0 < dif:
                res = res + pen*dif
    return res

In [74]:
o.optimiza(camara,20,0)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


array([  76.13349522,  -59.56833462,   26.77658047,  -71.23263302,
         43.18160134,  -85.32533915,    7.4305222 , -100.967819  ,
         27.67946709,  -94.91765915,   23.55157738, -126.81067639,
         16.8307711 , -102.91431736,   15.53264241, -101.94430247,
         40.02499441,  -95.16750386,   59.55528064,  -31.86100511])

Este resultado me emociona mucho, por cuestiones de tiempo fue imposible probar el problema con los números originales, pero aquí pruebo con 20 cámaras y 50 crímenes, así como un radio de 20 y una penalización de 5 escogidos de manera arbitraria, se resuelve usando BFGS. El problema se resolvió correctamente y da las coordenadas de las cámaras que no se centran todas en un mismo lugar tal y como queríamos.